In [60]:
import pandas as pd
import numpy as np
forecast_df=pd.read_csv('final_forecast_output.csv')
df=pd.read_csv('SparePartsInventory!.csv')

In [61]:
pd.set_option('display.max_columns',None)

In [62]:
import pandas as pd
import numpy as np



# Reusing the LeadTimeCalculator class logic
class LeadTimeCalculator:
    def __init__(self):
        self.base_lead_times = {'CA': 2.5, 'TX': 3.0, 'WI': 3.8}
        self.location_efficiency = {
            'CA_4': 0.9, 'CA_3': 1.0, 'CA_1': 1.1, 'CA_2': 1.15,
            'TX_1': 0.85, 'TX_2': 1.05, 'TX_3': 1.1,
            'WI_3': 1.0, 'WI_2': 1.05, 'WI_1': 1.2
        }
        self.criticality_multiplier = {
            'Service Critical': 0.8,
            'Operational Essential': 1.0,
            'Non-Critical': 1.3
        }
        self.volume_multiplier = {
            'High Volume': 0.9,
            'Medium Volume': 1.0,
            'Low Volume': 1.2
        }
        self.seasonal_adjustments = {
            'Early-Year Stock Reset': 1.3,
            'Spring Promotions': 1.1,
            'Summer Kickoff': 1.2,
            'Back-to-School': 1.15,
            'Christmas Peak': 1.4,
            'Pre-Holiday Promotions': 1.25,
            'Mother\'s Day & Summer Prep': 1.1
        }

    def calculate_base_lead_time(self, region, location_id, part_class, volume_class, season):
        base_lt = self.base_lead_times.get(region, 3.5)
        location_factor = self.location_efficiency.get(location_id, 1.0)
        criticality_factor = self.criticality_multiplier.get(part_class, 1.0)
        volume_factor = self.volume_multiplier.get(volume_class, 1.0)
        seasonal_factor = self.seasonal_adjustments.get(season, 1.0)
        return base_lt * location_factor * criticality_factor * volume_factor * seasonal_factor

    def apply_dynamic_adjustments(self, base_lt, is_weekend, is_event, snap_flag, is_month_end, is_payday, volatility_class):
        adjusted_lt = base_lt
        if is_weekend: adjusted_lt *= 1.15
        if is_event: adjusted_lt *= 1.2
        if snap_flag: adjusted_lt *= 1.1
        if is_month_end: adjusted_lt *= 1.08
        if is_payday: adjusted_lt *= 1.05
        volatility_adjustments = {
            'Highly Stable': 0.95,
            'Stable': 1.0,
            'Variable': 1.05,
            'Highly Volatile': 1.15
        }
        adjusted_lt *= volatility_adjustments.get(volatility_class, 1.0)
        return adjusted_lt

    def calculate_lead_time_distribution(self, mean_lt):
        std_dev = mean_lt * 0.25
        return {
            'mean': mean_lt,
            'std_dev': std_dev,
            'min': max(0.5, mean_lt * 0.6),
            'max': mean_lt * 2.0,
            'p90': mean_lt * 1.3,
            'p95': mean_lt * 1.5
        }

# Calculate lead time for df
calc = LeadTimeCalculator()
lead_time_results = []

for _, row in df.iterrows():
    base_lt = calc.calculate_base_lead_time(
        region=row['region'],
        location_id=row['location_id'],
        part_class=row['part_class'],
        volume_class=row['volume_class'],
        season=row['season']
    )
    adjusted_lt = calc.apply_dynamic_adjustments(
        base_lt=base_lt,
        is_weekend=row['is_weekend'],
        is_event=row['is_event'],
        snap_flag=row['snap_flag'],
        is_month_end=row['is_month_end'],
        is_payday=row['is_payday'],
        volatility_class=row['volatility_class']
    )
    dist = calc.calculate_lead_time_distribution(adjusted_lt)
    lead_time_results.append({
        'part_id': row['part_id'],
        'location_id': row['location_id'],
        'region': row['region'],
        'date': row['date'],
        'base_lead_time': round(base_lt, 2),
        'adjusted_lead_time': round(adjusted_lt, 2),
        **{f'lead_time_{k}': round(v, 2) for k, v in dist.items()}
    })

lead_time_df = pd.DataFrame(lead_time_results)
lead_time_df


,part_id,location_id,region,date,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95
0,BRAKE_PAD_1_005,TX_3,TX,2011-01-29,3.09,3.55,3.55,0.89,2.13,7.10,4.62,5.33
1,BRAKE_PAD_1_005,TX_3,TX,2011-01-30,3.09,3.54,3.54,0.89,2.13,7.09,4.61,5.31
2,BRAKE_PAD_1_005,TX_3,TX,2011-01-31,3.09,3.33,3.33,0.83,2.00,6.66,4.33,4.99
3,BRAKE_PAD_1_005,TX_3,TX,2011-02-01,2.61,2.87,2.87,0.72,1.72,5.74,3.73,4.30
4,BRAKE_PAD_1_005,TX_3,TX,2011-02-02,2.61,2.73,2.73,0.68,1.64,5.46,3.55,4.10
...,...,...,...,...,...,...,...,...,...,...,...,...
441738,LED_PANEL_2_149,WI_2,WI,2016-04-20,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74
441739,LED_PANEL_2_149,WI_2,WI,2016-04-21,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74
441740,LED_PANEL_2_149,WI_2,WI,2016-04-22,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74
441741,LED_PANEL_2_149,WI_2,WI,2016-04-23,6.22,8.23,8.23,2.06,4.94,16.46,10.70,12.35


In [63]:
forecast_df = forecast_df.rename(columns={'forecast_date': 'date'})

# Step 2: Merge on all required keys
forecast_df = forecast_df.merge(
    lead_time_df,
    on=['part_id', 'location_id', 'region', 'date'],
    how='left'
)

# Step 3: Restore the original column name
forecast_df = forecast_df.rename(columns={'date': 'forecast_date'})

In [64]:
import hashlib

# Compact forecast date: 2011-01-29 → 20110129
forecast_df['forecast_date_str'] = pd.to_datetime(forecast_df['forecast_date']).dt.strftime('%Y%m%d')

# Take first 3 letters from part_id
forecast_df['part_prefix'] = forecast_df['part_id'].str[:3].str.upper()

# Reuse the abbreviators
def abbreviate_part_type(x):
    return ''.join([word[0] for word in x.replace("_TYPE_", "_").split('_') if word])[:3].upper()

def abbreviate_demand_pattern(x):
    return x[:3].upper()

def clean_location(x):
    return x.replace("_", "").upper()

# Optional: if still not unique, use row number suffix
forecast_df = forecast_df.reset_index(drop=True)

forecast_df['customer_id'] = (
    "CUST_" +
    forecast_df['region'].str.upper() +
    forecast_df['location_id'].apply(clean_location) +
    forecast_df['part_type'].apply(abbreviate_part_type) +
    "_" +
    forecast_df['demand_pattern'].apply(abbreviate_demand_pattern) +
    "_" +
    forecast_df['ABC_Class'].str.upper() +
    forecast_df['XYZ_Class'].str.upper() +
    "_" +
    forecast_df['forecast_date_str'] +
    "_" +
    forecast_df['part_prefix'] +
    "_" +
    forecast_df.index.astype(str) 
)


In [65]:
forecast_df

,part_id,part_type,location_id,region,forecast_date,forecasted_demand,forecast_lower_bound,forecast_upper_bound,unit_cost,revenue,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,forecast_date_str,part_prefix,customer_id
0,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-29,2.000000,1.600000,2.400000,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Unknown,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.55,3.55,0.89,2.13,7.10,4.62,5.33,20110129,BRA,CUST_TXTX3BP1_LUM_AZ_20110129_BRA_0
1,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-30,2.100000,1.680000,2.520000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.54,3.54,0.89,2.13,7.09,4.61,5.31,20110130,BRA,CUST_TXTX3BP1_LUM_AZ_20110130_BRA_1
2,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-31,2.190000,1.752000,2.628000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.33,3.33,0.83,2.00,6.66,4.33,4.99,20110131,BRA,CUST_TXTX3BP1_LUM_AZ_20110131_BRA_2
3,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-01,2.171000,1.736800,2.605200,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.87,2.87,0.72,1.72,5.74,3.73,4.30,20110201,BRA,CUST_TXTX3BP1_LUM_AZ_20110201_BRA_3
4,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-02,1.953900,1.563120,2.344680,2.94,0.00,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.73,2.73,0.68,1.64,5.46,3.55,4.10,20110202,BRA,CUST_TXTX3BP1_LUM_AZ_20110202_BRA_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439867,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-20,0.240835,0.192668,0.289002,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74,20160420,LED,CUST_WIWI2LP2_LUM_CZ_20160420_LED_439867
439868,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-21,0.216752,0.173401,0.260102,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74,20160421,LED,CUST_WIWI2LP2_LUM_CZ_20160421_LED_439868
439869,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-22,0.195076,0.156061,0.234092,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74,20160422,LED,CUST_WIWI2LP2_LUM_CZ_20160422_LED_439869
439870,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-23,0.175569,0.140455,0.210682,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,8.23,8.23,2.06,4.94,16.46,10.70,12.35,20160423,LED,CUST_WIWI2LP2_LUM_CZ_20160423_LED_439870


In [66]:
import pandas as pd
import numpy as np
from scipy import stats


def calculate_robust_unit_level_inventory_metrics(forecast_df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds inventory metrics directly to the input DataFrame.

    Parameters:
        forecast_df (pd.DataFrame): SKU-location forecast data.

    Returns:
        pd.DataFrame: Same DataFrame with new columns added in-place.
    """
    # --- Safeguards ---
    safe_demand = forecast_df['forecasted_demand'].clip(lower=0.001)
    capped_rmse = forecast_df['rmse'].clip(upper=safe_demand * 2)
    min_holding_cost = 0.01

    # --- Cost & Service Levels ---
    forecast_df['service_level'] = np.where(forecast_df['ABC_Class'] == 'A', 0.98,
                                    np.where(forecast_df['ABC_Class'] == 'B', 0.95, 0.90))
    forecast_df['z_value'] = forecast_df['service_level'].apply(stats.norm.ppf)
    forecast_df['ordering_cost_per_order'] = np.where(forecast_df['ABC_Class'] == 'A', 800,
                                              np.where(forecast_df['ABC_Class'] == 'B', 600, 400))
    forecast_df['holding_cost_rate'] = np.where(forecast_df['volatility_class'] == 'Highly Volatile', 0.25,
                                         np.where(forecast_df['volatility_class'] == 'Variable', 0.22,
                                         np.where(forecast_df['volatility_class'] == 'Stable', 0.18, 0.15)))
    forecast_df['stockout_cost_per_unit'] = np.where(forecast_df['part_type'].str.contains('BRAKE_PAD', na=False), forecast_df['unit_cost'] * 5,
                                              np.where(forecast_df['part_type'].str.contains('FAN_MOTOR', na=False), forecast_df['unit_cost'] * 3,
                                                       forecast_df['unit_cost'] * 2))
    forecast_df['holding_cost_per_unit'] = (forecast_df['unit_cost'] * forecast_df['holding_cost_rate']).clip(lower=min_holding_cost)

    # --- Inventory Core ---
    forecast_df['demand_variance_during_lt'] = (capped_rmse ** 2) * forecast_df['lead_time_mean']
    forecast_df['safety_stock_units'] = forecast_df['z_value'] * np.sqrt(forecast_df['demand_variance_during_lt'])
    forecast_df['expected_demand_during_lt'] = safe_demand * forecast_df['lead_time_mean']
    forecast_df['reorder_point_units'] = forecast_df['expected_demand_during_lt'] + forecast_df['safety_stock_units']
    forecast_df['annual_demand_units'] = safe_demand * 365
    forecast_df['cv_demand'] = capped_rmse / safe_demand
    forecast_df['intermittent_adjustment'] = (1 + (forecast_df['cv_demand'] * 0.5)).clip(lower=1.0)
    forecast_df['eoq_units'] = np.sqrt(
        (2 * forecast_df['annual_demand_units'] * forecast_df['ordering_cost_per_order']) /
        forecast_df['holding_cost_per_unit']
    ) * forecast_df['intermittent_adjustment']
    forecast_df['cycle_stock_units'] = forecast_df['eoq_units'] / 2
    forecast_df['avg_inventory_level_units'] = forecast_df['safety_stock_units'] + forecast_df['cycle_stock_units']

    # --- Adjustments ---
    forecast_df['abc_xyz_safety_multiplier'] = np.where((forecast_df['ABC_Class'] == 'A') & (forecast_df['XYZ_Class'] == 'Z'), 1.3,
                                                np.where((forecast_df['ABC_Class'] == 'B') & (forecast_df['XYZ_Class'] == 'Z'), 1.2,
                                                np.where((forecast_df['ABC_Class'] == 'C') & (forecast_df['XYZ_Class'] == 'X'), 0.8, 1.0)))
    forecast_df['safety_stock_units_adjusted'] = forecast_df['safety_stock_units'] * forecast_df['abc_xyz_safety_multiplier']
    forecast_df['avg_inventory_level_units_adjusted'] = forecast_df['safety_stock_units_adjusted'] + forecast_df['cycle_stock_units']

    # --- Service Level Metrics ---
    forecast_df['expected_shortage_per_cycle'] = capped_rmse * np.sqrt(forecast_df['lead_time_mean']) * stats.norm.pdf(forecast_df['z_value'])
    forecast_df['fill_rate'] = 1 - (forecast_df['expected_shortage_per_cycle'] / forecast_df['expected_demand_during_lt'].replace(0, 0.001))
    forecast_df['days_of_supply'] = forecast_df['avg_inventory_level_units'] / safe_demand

    # --- Capping Output Columns ---
    max_units = 50000
    for col in [
        'safety_stock_units', 'reorder_point_units', 'eoq_units',
        'cycle_stock_units', 'avg_inventory_level_units',
        'safety_stock_units_adjusted', 'avg_inventory_level_units_adjusted'
    ]:
        forecast_df[col] = forecast_df[col].clip(0, max_units).fillna(0)
    forecast_df['fill_rate'] = forecast_df['fill_rate'].clip(0, 1).fillna(0)

    return forecast_df


calculate_robust_unit_level_inventory_metrics(forecast_df)

,part_id,part_type,location_id,region,forecast_date,forecasted_demand,forecast_lower_bound,forecast_upper_bound,unit_cost,revenue,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,forecast_date_str,part_prefix,customer_id,service_level,z_value,ordering_cost_per_order,holding_cost_rate,stockout_cost_per_unit,holding_cost_per_unit,demand_variance_during_lt,safety_stock_units,expected_demand_during_lt,reorder_point_units,annual_demand_units,cv_demand,intermittent_adjustment,eoq_units,cycle_stock_units,avg_inventory_level_units,abc_xyz_safety_multiplier,safety_stock_units_adjusted,avg_inventory_level_units_adjusted,expected_shortage_per_cycle,fill_rate,days_of_supply
0,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-29,2.000000,1.600000,2.400000,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Unknown,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.55,3.55,0.89,2.13,7.10,4.62,5.33,20110129,BRA,CUST_TXTX3BP1_LUM_AZ_20110129_BRA_0,0.98,2.053749,800,0.15,14.70,0.4410,4.792541,4.496041,7.100000,11.596041,730.000000,0.580950,1.290475,2100.156806,1050.078403,1054.574444,1.3,5.844853,1055.923257,0.105996,0.985071,527.287222
1,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-30,2.100000,1.680000,2.520000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.54,3.54,0.89,2.13,7.09,4.61,5.31,20110130,BRA,CUST_TXTX3BP1_LUM_AZ_20110130_BRA_1,0.98,2.053749,800,0.15,14.70,0.4410,4.779041,4.489704,7.434000,11.923704,766.500000,0.553286,1.276643,2128.953598,1064.476799,1068.966503,1.3,5.836615,1070.313415,0.105847,0.985762,509.031668
2,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-31,2.190000,1.752000,2.628000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.33,3.33,0.83,2.00,6.66,4.33,4.99,20110131,BRA,CUST_TXTX3BP1_LUM_AZ_20110131_BRA_2,0.98,2.053749,800,0.15,14.70,0.4410,4.495539,4.354499,7.292700,11.647199,799.350000,0.530548,1.265274,2154.734477,1077.367239,1081.721738,1.3,5.660849,1083.028087,0.102659,0.985923,493.936867
3,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-01,2.171000,1.736800,2.605200,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.87,2.87,0.72,1.72,5.74,3.73,4.30,20110201,BRA,CUST_TXTX3BP1_LUM_AZ_20110201_BRA_3,0.98,2.053749,800,0.15,14.70,0.4410,3.874533,4.042565,6.230770,10.273335,792.415000,0.535191,1.267596,2149.303550,1074.651775,1078.694340,1.3,5.255335,1079.907110,0.095305,0.984704,496.865196
4,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-02,1.953900,1.563120,2.344680,2.94,0.00,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.73,2.73,0.68,1.64,5.46,3.55,4.10,20110202,BRA,CUST_TXTX3BP1_LUM_AZ_20110202_BRA_4,0.98,2.053749,800,0.15,14.70,0.4410,3.685532,3.942733,5.334147,9.276880,713.173500,0.594657,1.297328,2086.835554,1043.417777,1047.360511,1.3,5.125554,1048.543331,0.092952,0.982574,536.035882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439867,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-20,0.240835,0.192668,0.289002,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74,20160420,LED,CUST_WIWI2LP2_LUM_CZ_20160420_LED_439867,0.90,1.281552,400,0.25,1.94,0.2425,1.661163,1.651742,1.724379,3.376121,87.904778,2.000000,2.000000,1077.023946,538.511973,540.163715,1.0,1.651742,540.163715,0.226193,0.868826,2242.878712
439868,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-21,0.216752,0.173401,0.260102,0.97,0.00

In [87]:
forecast_df

,part_id,part_type,location_id,region,forecast_date,forecasted_demand,forecast_lower_bound,forecast_upper_bound,unit_cost,revenue,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,forecast_date_str,part_prefix,customer_id,service_level,z_value,ordering_cost_per_order,holding_cost_rate,stockout_cost_per_unit,holding_cost_per_unit,demand_variance_during_lt,safety_stock_units,expected_demand_during_lt,reorder_point_units,annual_demand_units,cv_demand,intermittent_adjustment,eoq_units,cycle_stock_units,avg_inventory_level_units,abc_xyz_safety_multiplier,safety_stock_units_adjusted,avg_inventory_level_units_adjusted,expected_shortage_per_cycle,fill_rate,days_of_supply,achieved_service_level
0,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-29,2.000000,1.600000,2.400000,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Unknown,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.55,3.55,0.89,2.13,7.10,4.62,5.33,20110129,BRA,CUST_TXTX3BP1_LUM_AZ_20110129_BRA_0,0.98,2.053749,800,0.15,14.70,0.4410,4.792541,4.496041,7.100000,11.596041,730.000000,0.580950,1.290475,2100.156806,1050.078403,1054.574444,1.3,5.844853,1055.923257,0.105996,0.985071,527.287222,0.95
1,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-30,2.100000,1.680000,2.520000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.54,3.54,0.89,2.13,7.09,4.61,5.31,20110130,BRA,CUST_TXTX3BP1_LUM_AZ_20110130_BRA_1,0.98,2.053749,800,0.15,14.70,0.4410,4.779041,4.489704,7.434000,11.923704,766.500000,0.553286,1.276643,2128.953598,1064.476799,1068.966503,1.3,5.836615,1070.313415,0.105847,0.985762,509.031668,0.95
2,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-31,2.190000,1.752000,2.628000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.33,3.33,0.83,2.00,6.66,4.33,4.99,20110131,BRA,CUST_TXTX3BP1_LUM_AZ_20110131_BRA_2,0.98,2.053749,800,0.15,14.70,0.4410,4.495539,4.354499,7.292700,11.647199,799.350000,0.530548,1.265274,2154.734477,1077.367239,1081.721738,1.3,5.660849,1083.028087,0.102659,0.985923,493.936867,0.95
3,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-01,2.171000,1.736800,2.605200,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.87,2.87,0.72,1.72,5.74,3.73,4.30,20110201,BRA,CUST_TXTX3BP1_LUM_AZ_20110201_BRA_3,0.98,2.053749,800,0.15,14.70,0.4410,3.874533,4.042565,6.230770,10.273335,792.415000,0.535191,1.267596,2149.303550,1074.651775,1078.694340,1.3,5.255335,1079.907110,0.095305,0.984704,496.865196,0.95
4,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-02,1.953900,1.563120,2.344680,2.94,0.00,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.73,2.73,0.68,1.64,5.46,3.55,4.10,20110202,BRA,CUST_TXTX3BP1_LUM_AZ_20110202_BRA_4,0.98,2.053749,800,0.15,14.70,0.4410,3.685532,3.942733,5.334147,9.276880,713.173500,0.594657,1.297328,2086.835554,1043.417777,1047.360511,1.3,5.125554,1048.543331,0.092952,0.982574,536.035882,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439867,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-20,0.240835,0.192668,0.289002,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74,20160420,LED,CUST_WIWI2LP2_LUM_CZ_20160420_LED_439867,0.90,1.281552,400,0.25,1.94,0.2425,1.661163,1.651742,1.724379,3.376121,87.904778,2.000000,2.000000,1077.023946,538.511973,540.163715,1.0,1.651742,540.163715,0.226193,0.868826,2242.878712,0.95
439868,LED_PANEL_2_149,LED_PANEL_TYPE_

In [88]:
pd.set_option('display.max_rows',None)

In [89]:
pd.reset_option('display.max_rows')

In [122]:
def create_sku_level_inventory_summary_safe(unit_df):
    import numpy as np
    from scipy import stats

    def robust_revenue_agg(x):
        mode_val = x.mode()
        if not mode_val.empty and mode_val[0] > 0:
            return mode_val[0]
        x_nonzero = x[x > 0]
        if not x_nonzero.empty:
            return x_nonzero.median()
        return 0.0

    required_cols = [
        'unit_cost', 'revenue', 'annual_demand_units', 'avg_inventory_level_units_adjusted',
        'holding_cost_per_unit', 'ordering_cost_per_order', 'stockout_cost_per_unit',
        'eoq_units', 'lead_time_mean', 'forecasted_demand', 'achieved_service_level'
    ]
    for col in required_cols:
        if col not in unit_df.columns:
            if col in ['eoq_units', 'forecasted_demand', 'lead_time_mean']:
                unit_df[col] = 1.0
            elif col == 'achieved_service_level':
                unit_df[col] = 0.95
            else:
                unit_df[col] = 0.0

    for cat in ['ABC_Class', 'XYZ_Class', 'part_type', 'region', 'demand_pattern', 'replenishment_strategy']:
        if cat not in unit_df.columns:
            unit_df[cat] = 'Unknown'

    sku_summary = unit_df.groupby('part_id').agg({
        'unit_cost': 'mean',
        'revenue': robust_revenue_agg,
        'annual_demand_units': 'mean',
        'avg_inventory_level_units_adjusted': 'mean',
        'holding_cost_per_unit': 'mean',
        'ordering_cost_per_order': 'mean',
        'stockout_cost_per_unit': 'mean',
        'eoq_units': 'mean',
        'lead_time_mean': 'mean',
        'forecasted_demand': 'mean',
        'achieved_service_level': 'mean',
        'ABC_Class': lambda x: x.mode()[0] if not x.mode().empty else None,
        'XYZ_Class': lambda x: x.mode()[0] if not x.mode().empty else None,
        'part_type': lambda x: x.mode()[0] if not x.mode().empty else None,
        'region': lambda x: x.mode()[0] if not x.mode().empty else None,
        'demand_pattern': lambda x: x.mode()[0] if not x.mode().empty else None,
        'replenishment_strategy': lambda x: x.mode()[0] if not x.mode().empty else None
    }).reset_index()

    sku_summary['total_inventory_value'] = sku_summary['avg_inventory_level_units_adjusted'] * sku_summary['unit_cost']
    sku_summary['annual_holding_cost'] = sku_summary['total_inventory_value'] * sku_summary['holding_cost_per_unit'] / sku_summary['unit_cost']
    sku_summary['order_frequency'] = sku_summary['annual_demand_units'] / sku_summary['eoq_units']
    sku_summary['annual_ordering_cost'] = sku_summary['order_frequency'] * sku_summary['ordering_cost_per_order']
    sku_summary['annual_revenue'] = sku_summary['annual_demand_units'] * sku_summary['revenue']
    sku_summary['inventory_turnover'] = sku_summary['annual_demand_units'] / sku_summary['avg_inventory_level_units_adjusted']
    sku_summary['days_of_supply'] = sku_summary['avg_inventory_level_units_adjusted'] / sku_summary['forecasted_demand']
    sku_summary['gross_margin_per_unit'] = sku_summary['revenue'] - sku_summary['unit_cost']
    sku_summary['gross_margin_percent'] = (sku_summary['gross_margin_per_unit'] / sku_summary['revenue']).replace([np.inf, -np.inf], 0).fillna(0) * 100
    sku_summary['estimated_stockout_cost'] = sku_summary['stockout_cost_per_unit'] * (1 - sku_summary['achieved_service_level']) * sku_summary['forecasted_demand'] * 12
    sku_summary['total_annual_cost'] = sku_summary['annual_holding_cost'] + sku_summary['annual_ordering_cost'] + sku_summary['estimated_stockout_cost']

    sku_summary['contribution_margin'] = sku_summary['gross_margin_per_unit'] * sku_summary['annual_demand_units']
    sku_summary['value_contribution_ratio'] = sku_summary['contribution_margin'] / sku_summary['total_annual_cost']
    sku_summary['service_level_gap'] = 1.0 - sku_summary['achieved_service_level']
    sku_summary['inventory_value_rank'] = sku_summary['total_inventory_value'].rank(ascending=False)

    return sku_summary

# Usage
sku_summary = create_sku_level_inventory_summary_safe(forecast_df)


In [141]:
sku_summary


,part_id,unit_cost,revenue,annual_demand_units,avg_inventory_level_units_adjusted,holding_cost_per_unit,ordering_cost_per_order,stockout_cost_per_unit,eoq_units,lead_time_mean,forecasted_demand,achieved_service_level,ABC_Class,XYZ_Class,part_type,region,demand_pattern,replenishment_strategy,total_inventory_value,annual_holding_cost,order_frequency,annual_ordering_cost,annual_revenue,inventory_turnover,days_of_supply,gross_margin_per_unit,gross_margin_percent,estimated_stockout_cost,total_annual_cost,contribution_margin,value_contribution_ratio,service_level_gap,inventory_value_rank
0,BRAKE_PAD_1_005,3.327914,5.88,328.882382,633.238073,0.607609,800.0,16.639571,1258.277207,2.968348,0.900955,0.95,A,Z,BRAKE_PAD_TYPE_1,TX,Lumpy,Project-Based,2107.362022,384.761279,0.261375,209.100113,1933.828408,0.519366,702.852048,2.552086,43.402819,8.994903,602.856295,839.336034,1.392266,0.05,105.0
1,BRAKE_PAD_1_049,2.233413,2.24,134.599522,355.404658,0.467881,400.0,11.167067,708.517584,3.323602,0.368623,0.95,C,Z,BRAKE_PAD_TYPE_1,TX,Lumpy,Project-Based,793.765556,166.286947,0.189973,75.989376,301.502930,0.378722,964.140311,0.006587,0.294041,2.469865,244.746188,0.886542,0.003622,0.05,251.0
2,BRAKE_PAD_1_079,5.608291,5.74,25.747382,91.708712,1.184134,400.0,28.041455,182.862702,3.297855,0.070232,0.95,C,Z,BRAKE_PAD_TYPE_1,CA,Lumpy,Project-Based,514.329144,108.595406,0.140802,56.320687,147.789974,0.280752,1305.800514,0.131709,2.294582,1.181641,166.097734,3.391162,0.020417,0.05,280.0
3,BRAKE_PAD_1_083,1.979688,6.00,873.639712,1453.406868,0.355022,800.0,9.898441,2883.484374,3.570095,2.393524,0.95,A,Z,BRAKE_PAD_TYPE_1,WI,Lumpy,Project-Based,2877.292369,515.991855,0.302981,242.384448,5241.838273,0.601098,607.224626,4.020312,67.005197,14.215295,772.591599,3512.304078,4.546133,0.05,54.0
4,BRAKE_PAD_1_097,0.980000,1.96,629.609259,1559.412226,0.173311,600.0,4.900000,3108.436250,2.558438,1.724953,0.95,B,Z,BRAKE_PAD_TYPE_1,CA,Lumpy,Project-Based,1528.223981,270.262928,0.202549,121.529131,1234.034147,0.403748,904.031722,0.980000,50.000000,5.071362,396.863420,617.017074,1.554734,0.05,156.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,LED_PANEL_2_124,0.230000,0.46,149.826816,1205.236151,0.051750,400.0,0.460000,2405.836131,7.061683,0.410359,0.95,C,Z,LED_PANEL_TYPE_2,WI,Lumpy,Project-Based,277.204315,62.370971,0.062276,24.910560,68.920335,0.124313,2937.027497,0.230000,50.000000,0.113259,87.394790,34.460168,0.394305,0.05,290.0
286,LED_PANEL_2_125,0.962318,0.97,56.732785,327.921467,0.210101,400.0,1.924636,654.391687,6.620370,0.155273,0.95,C,Z,LED_PANEL_TYPE_2,TX,Lumpy,Project-Based,315.564811,68.896503,0.086695,34.678182,55.030801,0.173007,2111.899449,0.007682,0.791934,0.179307,103.753991,0.435807,0.004200,0.05,288.0
287,LED_PANEL_2_130,6.995614,6.97,73.910474,207.243132,1.603871,600.0,13.991228,410.668352,8.496079,0.202448,0.95,B,Z,LED_PANEL_TYPE_2,WI,Lumpy,Project-Based,1449.793002,332.391247,0.179976,107.985639,515.156006,0.356637,1023.686120,-0.025614,-0.367492,1.699497,442.076383,-1.893159,-0.004282,0.05,167.0
288,LED_PANEL_2_143,1.016574,1.17,51.508392,341.872676,0.239549,400.0,2.033148,682.225096,5.551597,0.141104,0.95,C,Z,LED_PANEL_TYPE_2,TX,Lumpy,Project-Based,347.538935,81.895135,0.075501,30.200233,60.264819,0.150665,2422.843050,0.153426,13.113318,0.172131,112.267499,7.902717,0.070392,0.05,287.0


In [124]:
import numpy as np

# Set reasonable upper bounds (adjust as needed for your business)
MAX_UNIT_COST = 10000
MAX_DEMAND = 100000
MAX_INVENTORY = 10000
MAX_INVENTORY_VALUE = 1_000_000
MAX_COST = 100_000

filtered = sku_summary[
    (sku_summary['unit_cost'] > 0) & (sku_summary['unit_cost'] < MAX_UNIT_COST) &
    (sku_summary['annual_demand_units'] > 0) & (sku_summary['annual_demand_units'] < MAX_DEMAND) &
    (sku_summary['avg_inventory_level_units_adjusted'] > 0) & (sku_summary['avg_inventory_level_units_adjusted'] < MAX_INVENTORY) &
    (sku_summary['total_inventory_value'] > 0) & (sku_summary['total_inventory_value'] < MAX_INVENTORY_VALUE) &
    (sku_summary['annual_holding_cost'] > 0) & (sku_summary['annual_holding_cost'] < MAX_COST) &
    (sku_summary['annual_ordering_cost'] > 0) & (sku_summary['annual_ordering_cost'] < MAX_COST)
].copy()

# Optionally, print how many rows remain
print(f"Filtered SKUs: {len(filtered)} out of {len(sku_summary)}")


Filtered SKUs: 290 out of 290


In [144]:
import pandas as pd
import numpy as np

def generate_opl_dat_file(
    sku_summary_df: pd.DataFrame,
    regional_budgets: dict,
    filename: str = "inventory_data.dat"
):
    """
    Generates a complete OPL .dat file from the sku_summary DataFrame.

    Parameters:
        sku_summary_df (pd.DataFrame): SKU-level summary containing cost and classification data.
        regional_budgets (dict): Dictionary with regional budget limits, e.g. {'TX': 750000}.
        filename (str): Name of the .dat file to be generated.
    """
    df = sku_summary_df.copy()
    
    # Ensure 'part_id' is index
    if 'part_id' in df.columns:
        df.set_index('part_id', inplace=True)
    
    # Define expected columns and map to OPL parameter names
    param_mapping = {
        'unit_cost': 'unit_cost',
        'annual_holding_cost': 'original_holding_cost',
        'annual_ordering_cost': 'original_ordering_cost',
        'estimated_stockout_cost': 'original_stockout_cost',
        'avg_inventory_level_units_adjusted': 'original_avg_inventory',
        'achieved_service_level': 'original_service_level',
        'ABC_Class': 'ABC_Class',
        'region': 'sku_to_region'
    }
    
    # Validate required columns
    missing_cols = [col for col in param_mapping if col not in df.columns]
    if missing_cols:
        raise KeyError(f"Missing required columns in DataFrame: {missing_cols}")

    with open(filename, "w") as f:
        # --- Write Sets ---
        f.write("SKUs = {\n" + ",\n".join(f'  "{sku}"' for sku in df.index) + "\n};\n\n")
        f.write("Regions = {\n" + ",\n".join(f'  "{r}"' for r in sorted(df['region'].unique())) + "\n};\n\n")

        # --- Write Parameters ---
        for df_col, opl_param in param_mapping.items():
            f.write(f"{opl_param} = #[\n")
            for sku, row in df.iterrows():
                val = row[df_col]
                if pd.isna(val):
                    continue  # Skip missing values
                if isinstance(val, str):
                    val_str = val.replace('"', '\\"')  # Escape quotes
                    f.write(f'  "{sku}": "{val_str}",\n')
                else:
                    f.write(f'  "{sku}": {float(val):.4f},\n')
            f.write("]#;\n\n")
        
        # --- Region-level Inventory Budget ---
        f.write("regional_inventory_budget = #[\n")
        for region, budget in regional_budgets.items():
            f.write(f'  "{region}": {float(budget):.2f},\n')
        f.write("]#;\n\n")

    print(f"✅ OPL .dat file '{filename}' successfully created.")

# ======================================================================
# EXAMPLE USAGE
# ======================================================================
regional_budgets_input = {
    'TX': 750000.0,
    'CA': 500000.0,
    'WI': 400000.0
}

generate_opl_dat_file(
    sku_summary_df=sku_summary,
    regional_budgets=regional_budgets_input,
    filename="inventory_data.dat"
)


✅ OPL .dat file 'inventory_data.dat' successfully created.


In [147]:
# Ensure batch size is defined
batch_size = 97

# Define regional budgets (replace or load dynamically as needed)
regional_budgets_input = {
    'TX': 750000.0,
    'CA': 500000.0,
    'WI': 400000.0
}

# Loop through DataFrame in batches of 50 rows
for i, start in enumerate(range(0, len(filtered), batch_size), start=1):
    batch_df = filtered.iloc[start:start + batch_size]
    output_filename = f"sku_data_batch_{i}.dat"
    
    # Generate the .dat file for this batch
    generate_opl_dat_file(
        sku_summary_df=batch_df,
        regional_budgets=regional_budgets_input,
        filename=output_filename
    )


✅ OPL .dat file 'sku_data_batch_1.dat' successfully created.
✅ OPL .dat file 'sku_data_batch_2.dat' successfully created.
✅ OPL .dat file 'sku_data_batch_3.dat' successfully created.


In [96]:
pd.reset_option('display.max_rows')

In [76]:
forecast_df['customer_id'].nunique()

439872

In [97]:
filtered.to_csv("final_inventory_optimization_output.csv")

In [132]:
import pulp
import pandas as pd
import numpy as np

def solve_optimization_from_summary_df(
    sku_summary_df: pd.DataFrame,
    total_inventory_budget: float = 750000.0
) -> pd.DataFrame:
    """
    Solves the enhanced inventory optimization problem using the pre-calculated
    sku_summary DataFrame as a direct input.

    This function incorporates stockout costs, service level targets, and a total
    inventory budget to find a realistic, business-focused optimal policy.

    Args:
        sku_summary_df (pd.DataFrame): The DataFrame generated by your summary function.
        total_inventory_budget (float): The maximum allowable total value for the optimized inventory.

    Returns:
        pd.DataFrame: A DataFrame containing the detailed optimization results for each SKU.
    """
    print("\n🚀 Starting Enhanced PuLP Optimization from DataFrame...")
    print("=" * 60)

    # --- Step 1: Prepare the Input DataFrame ---
    df = sku_summary_df.copy()
    if 'part_id' in df.columns:
        df.set_index('part_id', inplace=True)
    
    # --- CORRECTED: Map the columns from sku_summary to the names the model expects ---
    param_mapping = {
        'avg_inventory_level_units_adjusted': 'avg_inventory_level',
        'annual_holding_cost': 'original_holding_cost',
        'annual_ordering_cost': 'original_ordering_cost',
        'estimated_stockout_cost': 'original_stockout_cost', # Using your calculated stockout cost
        'achieved_service_level': 'fill_rate' # Using your calculated service level
    }
    df.rename(columns=param_mapping, inplace=True)

    # Validate that all necessary columns now exist after renaming
    required_cols = [
        'avg_inventory_level', 'original_holding_cost', 'original_ordering_cost',
        'original_stockout_cost', 'fill_rate', 'ABC_Class', 'unit_cost'
    ]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"DataFrame is missing required columns after renaming: {missing_cols}")

    df.dropna(subset=required_cols, inplace=True)
    skus = df.index.tolist()
    print(f"✅ Data prepared: {len(skus)} clean SKUs ready for optimization.")

    # --- Step 2: Declare the LP Model ---
    prob = pulp.LpProblem("Enhanced_Inventory_Optimization", pulp.LpMinimize)

    # --- Step 3: Define Decision Variables ---
    order_qty_factor = pulp.LpVariable.dicts("OrderQtyFactor", skus, lowBound=0.7, upBound=1.5)
    inventory_efficiency_factor = pulp.LpVariable.dicts("InventoryEfficiencyFactor", skus, lowBound=0.8, upBound=1.2)
    
    optimized_avg_inventory = pulp.LpVariable.dicts("OptimizedAvgInventory", skus, lowBound=0)
    optimized_holding_cost = pulp.LpVariable.dicts("OptimizedHoldingCost", skus, lowBound=0)
    optimized_ordering_cost = pulp.LpVariable.dicts("OptimizedOrderingCost", skus, lowBound=0)
    optimized_stockout_cost = pulp.LpVariable.dicts("OptimizedStockoutCost", skus, lowBound=0)
    optimized_total_cost = pulp.LpVariable.dicts("OptimizedTotalCost", skus, lowBound=0)
    achieved_service_level = pulp.LpVariable.dicts("AchievedServiceLevel", skus, lowBound=0, upBound=0.999)

    # --- Step 4: Define the Objective Function ---
    prob += pulp.lpSum([optimized_total_cost[s] for s in skus]), "Minimize_Total_Inventory_Cost"

    # --- Step 5: Add Comprehensive, Business-Relevant Constraints ---
    service_targets = {'A': 0.98, 'B': 0.95, 'C': 0.90}

    for s in skus:
        row = df.loc[s]
        prob += optimized_avg_inventory[s] == row['avg_inventory_level'] * inventory_efficiency_factor[s]
        prob += optimized_holding_cost[s] == row['original_holding_cost'] * inventory_efficiency_factor[s]
        prob += optimized_ordering_cost[s] == row['original_ordering_cost'] * order_qty_factor[s]
        prob += optimized_stockout_cost[s] == row['original_stockout_cost'] * (2.0 - inventory_efficiency_factor[s])
        prob += optimized_total_cost[s] == (optimized_holding_cost[s] + optimized_ordering_cost[s] + optimized_stockout_cost[s])
        prob += achieved_service_level[s] == row['fill_rate'] + 0.05 * (inventory_efficiency_factor[s] - 0.8) / 0.4
        prob += achieved_service_level[s] >= service_targets.get(row['ABC_Class'], 0.90)

    prob += pulp.lpSum([optimized_avg_inventory[s] * df.loc[s, 'unit_cost'] for s in skus]) <= total_inventory_budget, "Total_Inventory_Budget"

    # --- Step 6: Solve the Model ---
    print("🔧 Solving the enhanced optimization model...")
    prob.solve()
    print(f"✅ Status: {pulp.LpStatus[prob.status]}")

    if prob.status != pulp.LpStatusOptimal:
        print("❌ Optimization failed. The model may be infeasible or unbounded.")
        print("💡 Tip: Try increasing the `total_inventory_budget` or relaxing service level targets.")
        return None

    # --- Step 7: Extract and Format Comprehensive Results ---
    results = []
    for s in skus:
        row = df.loc[s]
        original_total = row['original_holding_cost'] + row['original_ordering_cost'] + row['original_stockout_cost']
        results.append({
            'SKU': s,
            'ABC_Class': row['ABC_Class'],
            'inventory_efficiency_factor': pulp.value(inventory_efficiency_factor[s]),
            'order_qty_factor': pulp.value(order_qty_factor[s]),
            'original_service_level': row['fill_rate'],
            'optimized_service_level': pulp.value(achieved_service_level[s]),
            'original_holding_cost': row['original_holding_cost'],
            'optimized_holding_cost': pulp.value(optimized_holding_cost[s]),
            'original_ordering_cost': row['original_ordering_cost'],
            'optimized_ordering_cost': pulp.value(optimized_ordering_cost[s]),
            'optimized_stockout_cost': pulp.value(optimized_stockout_cost[s]),
            'original_stockout_cost': row['original_stockout_cost'],
            'optimized_total_cost': pulp.value(optimized_ordering_cost[s]),
            'original_total_cost': original_total,
            'optimized_total_cost': pulp.value(optimized_total_cost[s]),
            'original_avg_inventory': row['avg_inventory_level'],
            'optimized_avg_inventory': pulp.value(optimized_avg_inventory[s])
        })

    results_df = pd.DataFrame(results)
    results_df['cost_savings_pct'] = ((results_df['original_total_cost'] - results_df['optimized_total_cost']) / results_df['original_total_cost'].replace(0, np.nan)) * 100
    
    output_file = 'pulp_enhanced_optimization_results.csv'
    results_df.to_csv(output_file, index=False)
    print(f"📁 Results saved to: {output_file}")
    
    return results_df 

# Now, you can call the optimization function with the DataFrame.
optimized_results = solve_optimization_from_summary_df(
    sku_summary_df=sku_summary,
    total_inventory_budget=750000.0
)
    
if optimized_results is not None:
    print("\n🎉 Enhanced PuLP optimization complete.")
    # ... (rest of summary printing)



🚀 Starting Enhanced PuLP Optimization from DataFrame...
✅ Data prepared: 290 clean SKUs ready for optimization.
🔧 Solving the enhanced optimization model...
✅ Status: Optimal
📁 Results saved to: pulp_enhanced_optimization_results.csv

🎉 Enhanced PuLP optimization complete.


In [133]:
optimized_results

,SKU,ABC_Class,inventory_efficiency_factor,order_qty_factor,original_service_level,optimized_service_level,original_holding_cost,optimized_holding_cost,original_ordering_cost,optimized_ordering_cost,optimized_stockout_cost,original_stockout_cost,optimized_total_cost,original_total_cost,original_avg_inventory,optimized_avg_inventory,cost_savings_pct
0,BRAKE_PAD_1_005,A,1.04,0.7,0.95,0.98,384.761279,400.151730,209.100113,146.370080,8.635107,8.994903,555.156920,602.856295,633.238073,658.56760,7.912230
1,BRAKE_PAD_1_049,C,0.80,0.7,0.95,0.95,166.286947,133.029560,75.989376,53.192563,2.963838,2.469865,189.185960,244.746188,355.404658,284.32373,22.701162
2,BRAKE_PAD_1_079,C,0.80,0.7,0.95,0.95,108.595406,86.876325,56.320687,39.424481,1.417969,1.181641,127.718770,166.097734,91.708712,73.36697,23.106254
3,BRAKE_PAD_1_083,A,1.04,0.7,0.95,0.98,515.991855,536.631530,242.384448,169.669110,13.646683,14.215295,719.947330,772.591599,1453.406868,1511.54310,6.813984
4,BRAKE_PAD_1_097,B,0.80,0.7,0.95,0.95,270.262928,216.210340,121.529131,85.070391,6.085634,5.071362,307.366370,396.863420,1559.412226,1247.52980,22.551096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,LED_PANEL_2_124,C,0.80,0.7,0.95,0.95,62.370971,49.896777,24.910560,17.437392,0.135911,0.113259,67.470080,87.394790,1205.236151,964.18892,22.798510
286,LED_PANEL_2_125,C,0.80,0.7,0.95,0.95,68.896503,55.117202,34.678182,24.274727,0.215168,0.179307,79.607097,103.753991,327.921467,262.33717,23.273220
287,LED_PANEL_2_130,B,0.80,0.7,0.95,0.95,332.391247,265.913000,107.985639,75.589948,2.039397,1.699497,343.542340,442.076383,207.243132,165.79451,22.288918
288,LED_PANEL_2_143,C,0.80,0.7,0.95,0.95,81.895135,65.516108,30.200233,21.140163,0.206557,0.172131,86.862828,112.267499,341.872676,273.49814,22.628696


In [79]:
forecast_df['customer_id'].nunique()

439872

In [80]:
sku_summary

,part_id,unit_cost,revenue,annual_demand_units,avg_inventory_level_units_adjusted,holding_cost_per_unit,ordering_cost_per_order,stockout_cost_per_unit,eoq_units,lead_time_mean,forecasted_demand,achieved_service_level,total_inventory_value,annual_holding_cost,order_frequency,annual_ordering_cost,annual_revenue,inventory_turnover,days_of_supply,gross_margin_per_unit,gross_margin_percent,estimated_stockout_cost,total_annual_cost
0,BRAKE_PAD_1_005,3.327914,2.982415,328.882382,633.238073,0.607609,800.0,16.639571,1258.277207,2.968348,0.900955,0.95,2107.362022,384.761279,0.261375,209.100113,980.863768,0.519366,702.852048,2.552086,85.571112,8.994903,602.856295
1,BRAKE_PAD_1_049,2.233413,0.793518,134.599522,355.404658,0.467881,400.0,11.167067,708.517584,3.323602,0.368623,0.95,793.765556,166.286947,0.189973,75.989376,106.807148,0.378722,964.140311,6.586587,830.048748,2.469865,244.746188
2,BRAKE_PAD_1_079,5.608291,0.377106,25.747382,91.708712,1.184134,400.0,28.041455,182.862702,3.297855,0.070232,0.95,514.329144,108.595406,0.140802,56.320687,9.709484,0.280752,1305.800514,3.211709,851.673471,1.181641,166.097734
3,BRAKE_PAD_1_083,1.979688,4.760920,873.639712,1453.406868,0.355022,800.0,9.898441,2883.484374,3.570095,2.393524,0.95,2877.292369,515.991855,0.302981,242.384448,4159.328483,0.601098,607.224626,3.900312,81.923496,14.215295,772.591599
4,BRAKE_PAD_1_097,0.980000,1.608303,629.609259,1559.412226,0.173311,600.0,4.900000,3108.436250,2.558438,1.724953,0.95,1528.223981,270.262928,0.202549,121.529131,1012.602220,0.403748,904.031722,-0.980000,-60.933806,5.071362,396.863420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,LED_PANEL_2_124,0.230000,0.092223,149.826816,1205.236151,0.051750,400.0,0.460000,2405.836131,7.061683,0.410359,0.95,277.204315,62.370971,0.062276,24.910560,13.817477,0.124313,2937.027497,3.050000,3307.201322,0.113259,87.394790
286,LED_PANEL_2_125,0.962318,0.146550,56.732785,327.921467,0.210101,400.0,1.924636,654.391687,6.620370,0.155273,0.95,315.564811,68.896503,0.086695,34.678182,8.314193,0.173007,2111.899449,-0.962318,-656.648102,0.179307,103.753991
287,LED_PANEL_2_130,6.995614,1.407726,73.910474,207.243132,1.603871,600.0,13.991228,410.668352,8.496079,0.202448,0.95,1449.793002,332.391247,0.179976,107.985639,104.045703,0.356637,1023.686120,-6.995614,-496.944277,1.699497,442.076383
288,LED_PANEL_2_143,1.016574,0.140227,51.508392,341.872676,0.239549,400.0,2.033148,682.225096,5.551597,0.141104,0.95,347.538935,81.895135,0.075501,30.200233,7.222842,0.150665,2422.843050,2.263426,1614.121305,0.172131,112.267499


In [81]:
forecast_df.columns

Index(['part_id', 'part_type', 'location_id', 'region', 'forecast_date',
       'forecasted_demand', 'forecast_lower_bound', 'forecast_upper_bound',
       'unit_cost', 'revenue', 'method_used', 'demand_pattern',
       'replenishment_strategy', 'ABC_Class', 'XYZ_Class', 'volume_class',
       'volatility_class', 'mae', 'rmse', 'mase', 'rmsse', 'bias',
       'avg_cost_impact', 'base_lead_time', 'adjusted_lead_time',
       'lead_time_mean', 'lead_time_std_dev', 'lead_time_min', 'lead_time_max',
       'lead_time_p90', 'lead_time_p95', 'forecast_date_str', 'part_prefix',
       'customer_id', 'service_level', 'z_value', 'ordering_cost_per_order',
       'holding_cost_rate', 'stockout_cost_per_unit', 'holding_cost_per_unit',
       'demand_variance_during_lt', 'safety_stock_units',
       'expected_demand_during_lt', 'reorder_point_units',
       'annual_demand_units', 'cv_demand', 'intermittent_adjustment',
       'eoq_units', 'cycle_stock_units', 'avg_inventory_level_units',
       'a

## CPLEX OPTIMIZED RESULT


In [151]:
import pandas as pd

# Load all 3 batches
df1 = pd.read_csv("Spare_parts_inventory1.csv")
df2 = pd.read_csv("Spare_parts_inventory2.csv")
df3 = pd.read_csv("Spare_parts_inventory3.csv")

# Combine all rows (vertical stacking)
df_combined = pd.concat([df1, df2, df3], ignore_index=True)

# Optional: Drop duplicate SKUs just in case (not needed if all are unique)
df_combined = df_combined.drop_duplicates(subset='SKU')
# Step 1: Rename 'SKU' → 'part_id'
df_combined.rename(columns={'SKU': 'part_id'}, inplace=True)

# Step 2: Merge on 'part_id'
df_combined = df_combined.merge(
    sku_summary[['part_id', 'annual_revenue']],
    on='part_id',
    how='inner'
)

# Step 3: Rename 'part_id' → 'SKU' again
df_combined.rename(columns={'part_id': 'SKU'}, inplace=True)

# Preview combined data
print(df_combined.to_markdown())

# Save final merged dataset
df_combined.to_csv("final_combined_SKUs.csv", index=False)


|     | SKU             | ABC_Class   |   inventory_efficiency_factor |   order_qty_factor |   original_service_level |   optimized_service_level |   original_holding_cost |   optimized_holding_cost |   original_ordering_cost |   optimized_ordering_cost |   original_stockout_cost |   optimized_stockout_cost |   original_total_cost |   optimized_total_cost |   original_avg_inventory |   optimized_avg_inventory |   cost_savings_pct |   annual_revenue |
|----:|:----------------|:------------|------------------------------:|-------------------:|-------------------------:|--------------------------:|------------------------:|-------------------------:|-------------------------:|--------------------------:|-------------------------:|--------------------------:|----------------------:|-----------------------:|-------------------------:|--------------------------:|-------------------:|-----------------:|
|   0 | BRAKE_PAD_1_005 | A           |                          1.04 |                0.7

In [152]:
Optimized_cplex=df_combined
Optimized_pulp=optimized_results

In [153]:
Optimized_pulp

,SKU,ABC_Class,inventory_efficiency_factor,order_qty_factor,original_service_level,optimized_service_level,original_holding_cost,optimized_holding_cost,original_ordering_cost,optimized_ordering_cost,optimized_stockout_cost,original_stockout_cost,optimized_total_cost,original_total_cost,original_avg_inventory,optimized_avg_inventory,cost_savings_pct
0,BRAKE_PAD_1_005,A,1.04,0.7,0.95,0.98,384.761279,400.151730,209.100113,146.370080,8.635107,8.994903,555.156920,602.856295,633.238073,658.56760,7.912230
1,BRAKE_PAD_1_049,C,0.80,0.7,0.95,0.95,166.286947,133.029560,75.989376,53.192563,2.963838,2.469865,189.185960,244.746188,355.404658,284.32373,22.701162
2,BRAKE_PAD_1_079,C,0.80,0.7,0.95,0.95,108.595406,86.876325,56.320687,39.424481,1.417969,1.181641,127.718770,166.097734,91.708712,73.36697,23.106254
3,BRAKE_PAD_1_083,A,1.04,0.7,0.95,0.98,515.991855,536.631530,242.384448,169.669110,13.646683,14.215295,719.947330,772.591599,1453.406868,1511.54310,6.813984
4,BRAKE_PAD_1_097,B,0.80,0.7,0.95,0.95,270.262928,216.210340,121.529131,85.070391,6.085634,5.071362,307.366370,396.863420,1559.412226,1247.52980,22.551096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,LED_PANEL_2_124,C,0.80,0.7,0.95,0.95,62.370971,49.896777,24.910560,17.437392,0.135911,0.113259,67.470080,87.394790,1205.236151,964.18892,22.798510
286,LED_PANEL_2_125,C,0.80,0.7,0.95,0.95,68.896503,55.117202,34.678182,24.274727,0.215168,0.179307,79.607097,103.753991,327.921467,262.33717,23.273220
287,LED_PANEL_2_130,B,0.80,0.7,0.95,0.95,332.391247,265.913000,107.985639,75.589948,2.039397,1.699497,343.542340,442.076383,207.243132,165.79451,22.288918
288,LED_PANEL_2_143,C,0.80,0.7,0.95,0.95,81.895135,65.516108,30.200233,21.140163,0.206557,0.172131,86.862828,112.267499,341.872676,273.49814,22.628696


In [154]:
Optimized_cplex

,SKU,ABC_Class,inventory_efficiency_factor,order_qty_factor,original_service_level,optimized_service_level,original_holding_cost,optimized_holding_cost,original_ordering_cost,optimized_ordering_cost,original_stockout_cost,optimized_stockout_cost,original_total_cost,optimized_total_cost,original_avg_inventory,optimized_avg_inventory,cost_savings_pct,annual_revenue
0,BRAKE_PAD_1_005,A,1.04,0.7,0.95,0.98,384.7613,400.151752,209.1001,146.37007,8.9949,8.635104,602.8563,555.156926,633.2381,658.567624,7.912229,1933.828408
1,BRAKE_PAD_1_049,C,0.80,0.7,0.95,0.95,166.2869,133.029520,75.9894,53.19258,2.4699,2.963880,244.7462,189.185980,355.4047,284.323760,22.701157,301.502930
2,BRAKE_PAD_1_079,C,0.80,0.7,0.95,0.95,108.5954,86.876320,56.3207,39.42449,1.1816,1.417920,166.0977,127.718730,91.7087,73.366960,23.106262,147.789974
3,BRAKE_PAD_1_083,A,1.04,0.7,0.95,0.98,515.9919,536.631576,242.3844,169.66908,14.2153,13.646688,772.5916,719.947344,1453.4069,1511.543176,6.813982,5241.838273
4,BRAKE_PAD_1_097,B,0.80,0.7,0.95,0.95,270.2629,216.210320,121.5291,85.07037,5.0714,6.085680,396.8634,307.366370,1559.4122,1247.529760,22.551092,1234.034147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,LED_PANEL_2_124,C,0.80,0.7,0.95,0.95,62.3710,49.896800,24.9106,17.43742,0.1133,0.135960,87.3949,67.470180,1205.2362,964.188960,22.798493,68.920335
286,LED_PANEL_2_125,C,0.80,0.7,0.95,0.95,68.8965,55.117200,34.6782,24.27474,0.1793,0.215160,103.7540,79.607100,327.9215,262.337200,23.273223,55.030801
287,LED_PANEL_2_130,B,0.80,0.7,0.95,0.95,332.3912,265.912960,107.9856,75.58992,1.6995,2.039400,442.0763,343.542280,207.2431,165.794480,22.288917,515.156006
288,LED_PANEL_2_143,C,0.80,0.7,0.95,0.95,81.8951,65.516080,30.2002,21.14014,0.1721,0.206520,112.2674,86.862740,341.8727,273.498160,22.628706,60.264819


In [162]:
import pandas as pd

# Ensure both DataFrames have consistent suffixes and merge
pulp_df = optimized_results.add_suffix('_pulp').rename(columns={'SKU_pulp': 'SKU'})
cplex_df = Optimized_cplex.add_suffix('_cplex').rename(columns={'SKU_cplex': 'SKU'})
merged = pulp_df.merge(cplex_df, on='SKU')

# Compute differences (PuLP - CPLEX)
diff_df = pd.DataFrame()
diff_df['SKU'] = merged['SKU']
diff_df['total_cost_diff'] = merged['optimized_total_cost_pulp'] - merged['optimized_total_cost_cplex']
diff_df['service_level_diff'] = merged['optimized_service_level_pulp'] - merged['optimized_service_level_cplex']
diff_df['holding_cost_diff'] = merged['optimized_holding_cost_pulp'] - merged['optimized_holding_cost_cplex']
diff_df['ordering_cost_diff'] = merged['optimized_ordering_cost_pulp'] - merged['optimized_ordering_cost_cplex']
diff_df['stockout_cost_diff'] = merged['optimized_stockout_cost_pulp'] - merged['optimized_stockout_cost_cplex']
diff_df['avg_inventory_diff'] = merged['optimized_avg_inventory_pulp'] - merged['optimized_avg_inventory_cplex']

# Suggest better optimizer based on lower total cost
diff_df['suggested_optimizer'] = diff_df['total_cost_diff'].apply(lambda x: 'CPLEX' if x > 0 else ('PuLP' if x < 0 else 'Same'))

# Output
diff_df


,SKU,total_cost_diff,service_level_diff,holding_cost_diff,ordering_cost_diff,stockout_cost_diff,avg_inventory_diff,suggested_optimizer
0,BRAKE_PAD_1_005,-0.000006,0.0,-0.000022,0.000010,0.000003,-0.000024,PuLP
1,BRAKE_PAD_1_049,-0.000020,0.0,0.000040,-0.000017,-0.000042,-0.000030,PuLP
2,BRAKE_PAD_1_079,0.000040,0.0,0.000005,-0.000009,0.000049,0.000010,CPLEX
3,BRAKE_PAD_1_083,-0.000014,0.0,-0.000046,0.000030,-0.000005,-0.000076,PuLP
4,BRAKE_PAD_1_097,0.000000,0.0,0.000020,0.000021,-0.000046,0.000040,Same
...,...,...,...,...,...,...,...,...
285,LED_PANEL_2_124,-0.000100,0.0,-0.000023,-0.000028,-0.000049,-0.000040,PuLP
286,LED_PANEL_2_125,-0.000003,0.0,0.000002,-0.000013,0.000008,-0.000030,PuLP
287,LED_PANEL_2_130,0.000060,0.0,0.000040,0.000028,-0.000004,0.000030,CPLEX
288,LED_PANEL_2_143,0.000088,0.0,0.000028,0.000023,0.000037,-0.000020,CPLEX


In [164]:
print("PuLP Columns:\n", pulp_df.columns.tolist())
print("CPLEX Columns:\n", cplex_df.columns.tolist())


PuLP Columns:
 ['SKU', 'ABC_Class_pulp_pulp', 'inventory_efficiency_factor_pulp_pulp', 'order_qty_factor_pulp_pulp', 'original_service_level_pulp_pulp', 'optimized_service_level_pulp_pulp', 'original_holding_cost_pulp_pulp', 'optimized_holding_cost_pulp_pulp', 'original_ordering_cost_pulp_pulp', 'optimized_ordering_cost_pulp_pulp', 'optimized_stockout_cost_pulp_pulp', 'original_stockout_cost_pulp_pulp', 'optimized_total_cost_pulp_pulp', 'original_total_cost_pulp_pulp', 'original_avg_inventory_pulp_pulp', 'optimized_avg_inventory_pulp_pulp', 'cost_savings_pct_pulp_pulp']
CPLEX Columns:
 ['SKU', 'ABC_Class_cplex_cplex', 'inventory_efficiency_factor_cplex_cplex', 'order_qty_factor_cplex_cplex', 'original_service_level_cplex_cplex', 'optimized_service_level_cplex_cplex', 'original_holding_cost_cplex_cplex', 'optimized_holding_cost_cplex_cplex', 'original_ordering_cost_cplex_cplex', 'optimized_ordering_cost_cplex_cplex', 'original_stockout_cost_cplex_cplex', 'optimized_stockout_cost_cplex_c

In [166]:
import pandas as pd

# Avoid double suffixing: just manually rename to _pulp and _cplex ONCE
pulp_df = Optimized_pulp.copy()
cplex_df = Optimized_cplex.copy()

# Remove any existing suffixes if needed (optional cleanup)
pulp_df.columns = [col.replace('_pulp_pulp', '') for col in pulp_df.columns]
cplex_df.columns = [col.replace('_cplex_cplex', '') for col in cplex_df.columns]

# Now apply suffixes cleanly
pulp_df = pulp_df.add_suffix('_pulp')
cplex_df = cplex_df.add_suffix('_cplex')

# Fix the SKU column for merge
pulp_df = pulp_df.rename(columns={'SKU_pulp': 'SKU'})
cplex_df = cplex_df.rename(columns={'SKU_cplex': 'SKU'})

# Merge on SKU
merged = pulp_df.merge(cplex_df, on='SKU')

# Sanity check
assert 'optimized_total_cost_pulp' in merged.columns
assert 'optimized_total_cost_cplex' in merged.columns

# Compare and suggest optimizer
merged['suggested_optimizer'] = (
    merged['optimized_total_cost_pulp'] - merged['optimized_total_cost_cplex']
).apply(lambda x: 'CPLEX' if x > 0 else ('PuLP' if x < 0 else 'Same'))

# Final Optimization Output Columns
columns = [
    'optimized_ordering_cost', 'optimized_holding_cost', 'optimized_stockout_cost',
    'optimized_total_cost', 'optimized_service_level', 'optimized_avg_inventory'
]

final_output = pd.DataFrame()
final_output['SKU'] = merged['SKU']
final_output['suggested_optimizer'] = merged['suggested_optimizer']

# Pick values based on suggested optimizer
for col in columns:
    final_output[col] = merged.apply(
        lambda row: row[f"{col}_pulp"] if row['suggested_optimizer'] == 'PuLP'
        else row[f"{col}_cplex"] if row['suggested_optimizer'] == 'CPLEX'
        else min(row[f"{col}_pulp"], row[f"{col}_cplex"]),
        axis=1
    )

# Optional: Add ABC class or other metadata
if 'ABC_Class_pulp' in merged.columns:
    final_output['ABC_Class'] = merged['ABC_Class_pulp']

# Final result
final_output


,SKU,suggested_optimizer,optimized_ordering_cost,optimized_holding_cost,optimized_stockout_cost,optimized_total_cost,optimized_service_level,optimized_avg_inventory,ABC_Class
0,BRAKE_PAD_1_005,PuLP,146.370080,400.151730,8.635107,555.156920,0.98,658.56760,A
1,BRAKE_PAD_1_049,PuLP,53.192563,133.029560,2.963838,189.185960,0.95,284.32373,C
2,BRAKE_PAD_1_079,CPLEX,39.424490,86.876320,1.417920,127.718730,0.95,73.36696,C
3,BRAKE_PAD_1_083,PuLP,169.669110,536.631530,13.646683,719.947330,0.98,1511.54310,A
4,BRAKE_PAD_1_097,Same,85.070370,216.210320,6.085634,307.366370,0.95,1247.52976,B
...,...,...,...,...,...,...,...,...,...
285,LED_PANEL_2_124,PuLP,17.437392,49.896777,0.135911,67.470080,0.95,964.18892,C
286,LED_PANEL_2_125,PuLP,24.274727,55.117202,0.215168,79.607097,0.95,262.33717,C
287,LED_PANEL_2_130,CPLEX,75.589920,265.912960,2.039400,343.542280,0.95,165.79448,B
288,LED_PANEL_2_143,CPLEX,21.140140,65.516080,0.206520,86.862740,0.95,273.49816,C
